# Solução dos problemas propostos n aula 02 - Interpolação trigonométrica


In [ ]:
using PyPlot
using FFTW

# Ortogonalidade discreta
Lembrando que 
\begin{align}
    2\cos\theta\cos\phi &= \cos(\theta-\phi) + \cos(\theta+\phi)\\
    2\cos\theta\sin\phi &= \sin(\theta+\phi) - \sin(\theta-\phi)\\
\end{align}

Chega-se à relação de ortogonalidade desejada:
$$
\sum_{i=0}^Q \phi_n(x_i)\cdot \phi_k(x_i)=0 \qquad k\ne n, N\: \text{não é múltiplo de}\: Q/2
$$

In [ ]:
function fourierseries(x, L0, a0, a, b)
    y = a0/2
    N = length(a)
    for i = 1:N
        y += a[i] * cos(2π*i*x/L0)
        y += b[i] * sin(2π*i*x/L0)
    end
    return y
end

function fourier_coeffs(N, x, y, L0)
    Q = length(x)
    a0 = 2/Q * sum(y)
    an = 2/Q * [sum(y .* cos.(2π .* n .* x ./ L0)) for n = 1:N]
    bn = 2/Q * [sum(y .* sin.(2π .* n .* x ./ L0)) for n = 1:N]
    return a0, an, bn
end

    
    
    

In [ ]:
#nfftutil(N) = (N % 2 == 0) ? (div(N,2) + 1) : div(N+1,2)
nfftutil(N) = div(N,2) + 1


## Interpolação trigonométrica

In [ ]:
struct TrigInterp
    Q::Int
    dx::Float64
    c::Vector{ComplexF64}
end
TrigInterp(x::Vector{Float64}, dx) = TrigInterp(length(x), dx, rfft(x))

function interp(f::TrigInterp, x)
    y = real(f.c[1])/2
    Q = f.Q
    P = Q * f.dx
    N = length(f.c)
    for k in 1:N-1
        c = f.c[k+1]
        y += real(c) * cos(2π*k*x/P) - imag(c) * sin(2π*k*x)
    end
    
    return 2y/Q
end

(f::TrigInterp)(x) = interp(f, x)

## Resample

In [ ]:
function myresample(x, qout)
    Q = length(x)
    X = rfft(x)
    Nutil = length(X)
    Nout = nfftutil(qout)
    X2 = zeros(ComplexF64, Nout)
    nmin = min(Nutil, Nout)
    for i in 1:nmin
        X2[i] = X[i] * qout/Q
    end
    
    return irfft(X2, qout)
end
   

# Problemas


## Problema 1

Seja a seguinte função definida no domínio $-1 < x \le 1$:

$$
y(x) = \left\{\begin{matrix} -1 & x < 0 \\ 1 & x > 0\\ 0 & x=0\end{matrix}\right.
$$

 1. Ache os coeficientes $a_n$ e $b_n$ da série de Fourier
 2. Plote a série de Fourier para diferentes números de termos da série de Fourier. Percebe algo estranho?
 3. Qual a relação desta função com a função $y = |x|$ que estudamos anteriormente?
 4. Escolha diferentes números de pontos e interpole esta função nestes pontos

### Solução

#### Ache os coeficientes $𝑎_n$ e $𝑏_𝑛$ da série de Fourier

Esta é uma função ímpar, portanto os coeficientes $a_n$ são nulos!

$$
b_n = \frac{2}{L_0}\int_{-L_0/2}^{L_0/2} y(x) \cdot \sin\left(\frac{2\pi n x}{L_0}\right)\:dx = 2\int_0^1 \sin\left(\pi n x\right)\:dx = \frac{2}{n\pi}\left(1 - \cos n\pi\right) = \frac{2}{n\pi}\left[1 - (-1)^n\right]
$$


In [ ]:
x = range(-1.0, 1.0, length=201);
y0 = 1.0 .* sign.(x)
bn = [2/(π*n) * (1.0 - cos(π*n)) for n in 1:500];

In [ ]:
sinseries(x, b, N, L0) = sum(b[n]*sin(2π*n*x/L0) for n in 1:N);

#### Plote a série de Fourier para diferentes números de termos da série de Fourier

In [ ]:
plot(x, y0, "k:")
for n in 1:2:15
    plot(x, sinseries.(x, Ref(bn), n, 2.0), label="n = $n")
end
legend()

In [ ]:
x1 = range(-1, 1, length=1001)
plot(x1, sinseries.(x1, Ref(bn), 30, 2.0))

#### Qual a relação desta função com a função $y = |x|$ que estudamos anteriormente?

Esta função é a derivada da função $y = |x|$!

E a série de $dy/dx$ é formada pelas derivadas da série original!

Existem condições específicas onde se pode derivar a série termo a termo. Mas se as funções forem suaves, derive e integre a vontade! Funciona até para algumas descontinuidades.




#### Escolha diferentes números de pontos e interpole esta função nestes pontos

In [ ]:
function myresample1(x, qout)
    Q = length(x)
    X = rfft(x)/Q
    Nutil = length(X)
    Nout = nfftutil(qout)
    X2 = zeros(ComplexF64, Nout)
    nmin = min(Nutil, Nout)
    for i in 1:nmin
        X2[i] = X[i] 
    end
    
    return irfft(X2, qout)
end
   

In [ ]:
Q0 = 4
x0 = range(-1, 1, length=Q0+1)[1:Q0]
y0 = 1.0 * sign.(x0);

In [ ]:
q = length(y0)
Y = rfft(y0)/q
Nutil = length(Y)
Nout = nfftutil(8)
Y2 = zeros(ComplexF64, Nout)
for i in 1:Nutil
    Y2[i] = Y[i]
end
Y2

In [ ]:
y2 = brfft(Y2, 8)

In [ ]:
rfft(y2)/8

In [ ]:
Q1 = 8
x1 = range(-1, 1, length=Q1+1)[1:Q1]
y1 = myresample1(y0, Q1)
plot(x1, y1)
plot(x0, y0, "r.")


## Problema 2

Verifique a ortogonalidade discreta da base trigonométrica, ou seja, 

$$
\sum_{i=0}^Q \phi_n(x_i)\cdot \phi_k(x_i)=0 \qquad k\ne n, N\: \text{não é múltiplo de}\: Q/2
$$

para 

\begin{align}
\phi_0(x) &= \frac{1}{2}\\
\phi_n(x) &= \cos\left(\frac{2\pi n x}{L_0}\right) \qquad 0\le n \le N \\
\phi_{n+N}(x) &= \sin\left(\frac{2\pi n x}{L_0}\right) \qquad 1\le n \le N 
\end{align}



## Problema 3

Nos exemplos iniciais usando a DFT, ao se escolher os pontos, foi utilizado o seguinte código:
```julia
L₀ = 1.0; Q = 16; freq=1
x = range(0.0, L₀, length=Q+1)[1:Q]
```
O que acontece se utilzarmos a expressão mais simples a seguir?
```julia
x = range(0.0, L₀, length=Q)
```
O que está acontecendo? Você consegue explicar?

In [ ]:
L₀ = 1.0; Q = 16; freq=1; 
x = range(0.0, L₀, length=Q)
y = sin.(freq.*2π.*x); f = (0:Q-1) / L₀
Y = 1/Q * fft(y);
#plot(f, real.(Y), "xb")
#plot(f, imag.(Y), "ro")

## Problema 4

Implemente a DFT usando a definição. Verifique como aumenta o custo computacional quando o número de pontos aumenta. 

Como foi mostrado, esta operação é uma multiplicação de matriz por vetor. Verifique se invertendo a ordem dos loops afeta o desempenho. 

## Problema 5

(Desafio) procure na net ou em algum livro como implementar a FFT. Tente!

## Problema 6

A transformada de Fourier está diretamente relacionada com a série de Fourier e é dada por:

$$
X(\omega) = \frac{1}{2\pi} \int_{-\infty}^\infty x(t)e^{-i\omega t}\: dt
$$

 1. Calcula a transformada de Fourier para a função 
  $$
  x(t) = \left\{\begin{matrix} 1 & -1 < x < 1 \\ 0 & x \ge 1\end{matrix}\right.
  $$
 2. Discretize a função e use a FFT para calcular a transformada.
 3. Tente verificar o que acontece se você adiciona zeros à esquerda e à direita da função discretizada.


## Problema 7

Faça alguns exemplos do pacote `ApproxFun` (<https://github.com/JuliaApproximation/ApproxFun.jl>)
